# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import string
from matplotlib.patches import Rectangle
from ipywidgets import interact, widgets
import os
from scipy.interpolate import interp1d, griddata
from scipy.signal import savgol_filter
from scipy.interpolate import make_interp_spline

# Config

In [2]:
config = {}

# ROMY coordinates
config['ROMY_lon'] = 11.275501
config['ROMY_lat'] = 48.162941

# path for figures to store
config['outpath_figs'] = "C:/Bachelorarbeit/figures/Geschwindigkeit/ROMYRLAS/"
# path for output data
config['outpath_data'] = "C:/Bachelorarbeit/data/waveformsROMYRLAS/"


In [3]:
outpath = config['outpath_figs']
outpath_FUR ="C:/Bachelorarbeit/figures/Geschwindigkeit/FURWET/"
df_Z = pd.read_csv(outpath_FUR + "spektralanalyse_ergebnisse_Z.csv")
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")
font_title=17
font_ax=13
font_leg=11

# Hilfsfunktionen


In [4]:
def plot_component_periode(ax, df, title):
    ratio_cols = [col for col in df.columns if col.startswith("ratio_")]
    frequencies = [float(col.split("_")[1][:-2]) for col in ratio_cols]
    periods = [1.0 / f for f in frequencies]
    # Plot nach Tiefenbereichen
    for (dmin, dmax), label in zip(depth_bins, depth_labels):
        df_subset = df[(df["depth"] >= dmin) & (df["depth"] < dmax)]
        num_events = len(df_subset)
        if df_subset.empty:
            continue

        mean_ratios = df_subset[ratio_cols].mean()
        sem_ratios = df_subset[ratio_cols].sem()
        legend_label = f"depth: {label} (n = {num_events})"

        if dmin == 0 and dmax == 700:
            ax.plot(periods, mean_ratios.values, linestyle='--', linewidth=2, label=legend_label)
        else:
            ax.plot(periods, mean_ratios.values, linestyle='-', label=legend_label)

        ax.fill_between(periods,
                        mean_ratios.values - sem_ratios.values,
                        mean_ratios.values + sem_ratios.values,
                        alpha=0.2)

    # Gesamtmittelwert über alle Tiefen (ohne SEM)
    overall_mean = df[ratio_cols].mean()
    ax.plot(periods,
            overall_mean.values,
            color='black',
            linestyle=':',
            linewidth=2,
            label='Overall mean (all depths)')

    ax.set_xlabel("Period [s]",fontsize=font_ax)
    ax.set_ylabel("Mean ratio (ROMY/RLAS)",fontsize=font_ax)
    ax.set_title(title,fontsize=font_title)
    ax.grid(True)
    ax.legend(fontsize=font_leg)

# Funktion zur Vorbereitung der Daten
def prepare_plot_data_periode(df):
    ratio_cols = [col for col in df.columns if col.startswith("ratio_")]
    frequencies = [float(col.split("_")[1][:-2]) for col in ratio_cols]
    periods = [1.0 / f for f in frequencies]
    return ratio_cols, periods

def smooth_line_periode(x, y, points=500):
    x = np.array(x)
    y = np.array(y)
    x_new = np.linspace(x.min(), x.max(), points)
    spline = make_interp_spline(x, y, k=4)  # kubische Interpolation
    y_smooth = spline(x_new)
    return x_new, y_smooth

# Funktion zur Erstellung der Interpolationsdaten

def plot_component_with_sem_periode(ax, df, color, label):
    # Frequenzen und Perioden extrahieren
    ratio_cols = [col for col in df.columns if col.startswith("ratio_")]
    frequencies = [float(col.split("_")[1][:-2]) for col in ratio_cols]
    periods = [1.0 / f for f in frequencies]  # Umrechnung in Periode

    # Mittelwert und SEM berechnen
    mean_ratios = df[ratio_cols].mean()
    sem_ratios = df[ratio_cols].sem()
    num_events = len(df)

    # Arrays erstellen
    period_array = np.array(periods)
    mean_array = mean_ratios.values
    sem_array = sem_ratios.values

    # Nach Perioden sortieren
    sort_idx = np.argsort(period_array)
    period_array = period_array[sort_idx]
    mean_array = mean_array[sort_idx]
    sem_array = sem_array[sort_idx]

    # Plot ohne Glättung
    ax.plot(period_array, mean_array, color=color, linewidth=2, label=f"{label} (n = {num_events})")
    ax.fill_between(period_array,
                    mean_array - sem_array,
                    mean_array + sem_array,
                    color=color,
                    alpha=0.3,
                    label=f"{label} ± SEM")


## Verhältnis gegen Frequenz

In [5]:
fig, ax = plt.subplots(figsize=(10, 6))

df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")

plot_component_with_sem_periode(ax, df_Z, color='green', label='Vertical component')

# Achsen und Titel
ax.set_xlabel("Periode (s)",fontsize=font_ax)
ax.set_xscale('log')
ax.set_ylabel("Mean ratio (ROMY/RLAS)",fontsize=font_ax)
ax.set_title("Mean ratio per frequency (± SEM)",fontsize=font_title)
ax.grid(True)
ax.legend(loc='best', fontsize=font_leg)

# Layout & speichern
plt.tight_layout()
plt.savefig(outpath + "meanratio_freqency_TandZ_SEM_ROMYRLAS_periode.png", dpi=300)
plt.close()


In [6]:
fig, ax = plt.subplots(figsize=(10, 6))
df_T = pd.read_csv(outpath_FUR + "spektralanalyse_ergebnisse_Z.csv")
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")
df_TF=pd.read_csv(outpath_FUR + "spektralanalyse_ergebnisse_T.csv")
# Komponenten plotten
plot_component_with_sem_periode(ax, df_T, color='blue', label='FUR/WET (Z)')
plot_component_with_sem_periode(ax, df_TF, color='orange', label='FUR/WET (T)')
plot_component_with_sem_periode(ax, df_Z, color='green', label='ROMY/RLAS (Z)')

# Achsen und Titel
ax.set_xlabel("Periode (s)",fontsize=font_ax)
ax.set_xscale('log')
ax.set_xlim(0.5, 10.5)
ax.set_ylabel("Mean ratio (ROMY/RLAS)",fontsize=font_ax)
ax.set_title(f"Mean ratio per frequency (± SEM) \n comparison FUR/WET and ROMY/RLAS",fontsize=font_title)
ax.grid(True)
ax.legend(loc='best', fontsize=font_ax)

# Layout & speichern
plt.tight_layout()
plt.savefig(outpath + "TZZmeanratio_freqency_TandZ_SEM_ROMYRLAS_periode_comparison.png", dpi=300)
plt.close()


In [7]:
fig, ax = plt.subplots(figsize=(10, 6))
df_T = pd.read_csv(outpath_FUR + "spektralanalyse_ergebnisse_Z.csv")
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")

# Komponenten plotten
plot_component_with_sem_periode(ax, df_T, color='blue', label='FUR/WET (Z)')
plot_component_with_sem_periode(ax, df_Z, color='green', label='ROMY/RLAS (Z)')

# Achsen und Titel
ax.set_xlabel("Periode (s)",fontsize=font_ax)
ax.set_xscale('log')
ax.set_xlim(0.5, 10.5)
ax.set_ylabel("Mean ratio (ROMY/RLAS)",fontsize=font_ax)
ax.set_title(f"Mean ratio per frequency (± SEM) \n comparison FUR/WET and ROMY/RLAS",fontsize=font_title)
ax.grid(True)
ax.legend(loc='best', fontsize=font_leg)

# Layout & speichern
plt.tight_layout()
plt.savefig(outpath + "meanratio_freqency_TandZ_SEM_ROMYRLAS_periode_comparison.png", dpi=300)
plt.close()


# Tiefen


### Verhältnis gegen Frequenz unterteilt in Tiefengruppen

In [8]:
# Konfiguration
depth_bins = [(0, 100), (100, 700)]
depth_labels = ["0–100 km", "100–700 km"]
full_range = (0, 700)
df_T = pd.read_csv(outpath_FUR + "spektralanalyse_ergebnisse_Z.csv")
df_Z = pd.read_csv(outpath + "spektralanalyse_ergebnisse_Z.csv")

In [13]:
# Subplots erstellen
fig, axs = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

# Komponenten plotten
plot_component_periode(axs[0], df_T, "FUR/WET (Z)")
plot_component_periode(axs[1], df_Z, "ROMY/RLAS (Z)")
axs[0].text(0.02, 0.95, "a)", transform=axs[0].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
axs[1].set_xscale('log')
axs[0].set_xscale('log')
axs[1].text(0.02, 0.95, "b)", transform=axs[1].transAxes,
            fontsize=14, fontweight='bold', va='top', ha='left')
# Formatierung & Speichern
plt.suptitle("Mean ratio (ROMY/RLAS) against period divided by depth range (± SEM)",fontsize=font_title)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig(outpath + "meanratio_freqency_depths_ROMYRLASundFURWET_periode.png", dpi=300)

plt.close()

In [11]:
# Subplots erstellen
fig, axs = plt.subplots(1, 1, figsize=(10, 6), sharex=True)

# Komponenten plotten

plot_component_periode(axs, df_Z, "ROMY/RLAS (Z)")
axs.set_xscale('log')
# Formatierung & Speichern
plt.suptitle("Mean ratio (ROMY/RLAS) against period divided by depth range (± SEM)",fontsize=font_title)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig(outpath + "meanratio_depths_ROMYRLAS_periode.png", dpi=300)

plt.close()